In [32]:
import json
import numpy as np
from itertools import groupby
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from collections import Counter

In [2]:
class News:
    def __init__(self, id, date, title, content, url, siteType):
        self.id = id
        self.date = date
        self.title = title
        self.content = content
        self.url = url
        self.siteType = siteType
    
    @classmethod
    def from_json(cls, json_str):
        json_dict = json.loads(json_str)
        return cls(**json_dict)

In [3]:
news = []
with open('/data/kasandra/year/all.normalized.json', encoding="utf8") as f:
    for line in f:
        news.append(News.from_json(line))

In [4]:
words = []
for n in news:
    words.extend(n.content.split())
counts = Counter(words)
one_time = [k for k, v in dict(counts).items() if v == 1]
print("total words: %s" % (len(words) - len(one_time)))

total words: 47479434


In [5]:
stopwords = set(one_time)

In [35]:
def zip_news(n,l):
    return list(map(assign_label_to_news, zip(n, l)))

def assign_label_to_news(tuplezz):
    (nws, lbl) = tuplezz
    nws.label = lbl.item()
    return nws

def filter_words(text):
    words_list = text.split()
    newWords = [x for x in words_list if len(x) > 3]
    return " ".join(newWords)

def print_clusters(cluster_news, clustre_labels):
    newsLabels = zip_news(cluster_news, clustre_labels)
    newsLabels = sorted(newsLabels, key=lambda n: n.label)
    for label, group in groupby(newsLabels, lambda n: n.label):
        groupList = list(group)
        print("Cluster: %s, count news: %s, titles:" % (label, len(groupList)))
        for gr in groupList:
            print("\t" + gr.title)

# Март 2015

In [7]:
mart_start = 1425157200000
mart_end = 1427835600000

In [8]:
mart_news = list(filter(lambda x: x.date > mart_start and x.date < mart_end, news))
mart_content = [filter_words(x.content) for x in mart_news]
print("count mart news: %s" % len(mart_content))

count mart news: 2684


In [9]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True, tokenizer=lambda text: text.split(" "), stop_words=stopwords) # , ngram_range=(1, 3)
tfidf_matrix = tfidf_vectorizer.fit_transform(mart_content)
print("vocabulary size: %s" % len(tfidf_vectorizer.vocabulary_))

vocabulary size: 36964


In [10]:
# X = (tfidf_matrix * tfidf_matrix.T).A

## DBScan

In [30]:
db = DBSCAN(eps=1.12, min_samples=3, n_jobs=-1).fit(tfidf_matrix)
# 1.36 for (1, 3)
# 1.12 for 1

print('count clusters: %d' % (len(set(db.labels_)) - (1 if -1 in db.labels_ else 0)))

labels = db.labels_
labels[:100]
print("-1: %s, 0: %s" % (labels.tolist().count(-1), labels.tolist().count(0)))

count clusters: 102
-1: 1547, 0: 227


In [31]:
print_clusters(mart_news, labels)

Cluster: -1, count news: 1547, titles:
	Американский астронавт почтил память Леонарда Нимоя в космосе
	Власти Севастополя национализировали завод Порошенко
	Что изменилось с 1 марта: Кредиты - в хорошие руки, лекарства - под госконтроль
	Лондон заблокировал сделку по покупке фондом Фридмана «дочки» RWE
	ЦБ назвал «немного чрезмерным» прогноз Moody's по инфляции в 22%
	С 1 марта россияне смогут въехать на Украину только по загранпаспорту
	В Севастополе национализирован завод Порошенко
	До поздней ночи в Москве несли цветы к месту гибели Бориса Немцова
	Путеводитель по удивительным местам России: три самые необычные пещеры
	20 лет назад не стало Владислава Листьева
	Со дня гибели Владислава Листьева исполнилось 20 лет
	Два кота в одном доме: как сделать из них дружную банду?
	Турецкий суд обязал мужчину заплатить жене за нелюбовь
	Хью Джекман покатался на велосипеде по Красной площади и рассказал о своих "секретах красоты"
	Charlie Hedbo оскандалился с карикатурой про Донбасс
	В России м

# KMeans

In [39]:
num_clusters = 100

km = KMeans(n_clusters=num_clusters, max_iter=300, n_init=30, n_jobs=-1).fit(tfidf_matrix)

clusters = km.cluster_centers_
labels = km.labels_
labels[:100]

array([78, 26, 13, 54, 10, 15, 55, 15, 26, 55, 21, 15,  3,  0, 78, 61, 67,
       37,  2, 47, 15, 35, 15, 87, 15, 47, 37, 37, 15, 92,  8, 83,  2, 88,
       86, 83, 89,  8, 37, 15, 37,  2, 47,  8, 15,  5, 60, 55, 37, 30, 15,
       55,  8, 55, 35, 37,  8, 15, 87, 83, 39, 35, 99, 13, 41, 35, 86, 40,
       91,  8, 37, 15, 55,  5, 47, 15, 47, 45, 10, 97,  4, 61, 71, 87,  2,
       42, 61, 35, 10, 87, 83,  8, 67, 55, 15, 15,  2,  4, 37, 67], dtype=int32)

In [40]:
print_clusters(mart_news, labels)

Cluster: 0, count news: 17, titles:
	20 лет назад не стало Владислава Листьева
	Фейк-долгожитель: СМИ опять поверили, что Пол Маккартни умер 50 лет назад
	Украинцы просят не оружие, а иностранных солдат, которые сметут с лица земли жителей Донбасса
	Как заработать паяльником
	«Яндекс» опубликовал список самых непонятных пользователям слов
	Спецкор «Комсомолки» помог женщине из Львова разыскать мужа - украинского солдата, погибшего в Донбассе
	Евросоюз объявил войну российским СМИ: план действий нарисует Могерини
	Скандал в украинской моде: супермодель Снежана Онопко обвинила редактора киевского «Vogue» в непатриотизме
	Ирада Зейналова: Снять бы туфли - и в командировку
	«Не ради правды, а в интересах истины»*, или казус Андрея Бабицкого
	Почему Киев борется с журналистами. Реплика Максима Кононенко
	Польскому журналисту заткнули рот. В прямом смысле
	Шарль Азнавур пообещал встретиться с пожилой поклонницей из России
	Владелец китайского ресторана в Кении арестован за сегрегацию
	В Швец